<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train1_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:

train_1 = pd.read_csv(r'/content/train_data1.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [ ]:

train_1['Target'] = train_1['Target'].replace(['Others'],'Invalid')


In [ ]:
train_1['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [ ]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train1_Xtfidf = tfidf_vect.fit_transform(train_1['Sentence'])


In [ ]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [ ]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [ ]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [ ]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [ ]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [ ]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train1_Xtfidf,train_1['Target'],test_size=0.26,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6592920353982301
                  precision    recall  f1-score   support

        Analysis       0.48      0.29      0.36       126
      Conclusion       0.81      0.48      0.60        46
           Facts       0.71      0.88      0.79       485
         Invalid       0.51      0.55      0.53       132
           Issue       0.60      0.47      0.53        51
Rule/Law/Holding       0.64      0.25      0.36        64

        accuracy                           0.66       904
       macro avg       0.63      0.49      0.53       904
    weighted avg       0.65      0.66      0.64       904



In [ ]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [ ]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 3, 3, 2, 3, 0, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 2, 2, 0, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2]
[0.9983895681441433, 0.9945912794661442, 0.9986410647263183, 0.9987993632599621, 0.9986669534326585, 0.9883929020864957, 0.9963544582939021, 0.9849982654945296, 0.9936806339291876, 0.9961597401312617, 0.9982680823352607, 0.9961696973918801, 0.9963894091821727, 0.9987768066744133, 0.9981218514809059, 0.9810586276984151, 0.9985765234817896, 0.9957093536492092, 0.9958025704485411, 0.9977454906314085, 0.9984870007798121, 0.9912770306121645, 0.9929279538182413, 0.993569458495193, 0.993248929267554, 0.9854873493817615, 0.9973149900620633, 0.991751152716523, 0.9914812812293273, 0.9954709638030056, 0.9959771255857377, 0.9821919356384237, 0.9960257464934157, 0.9937406609072525, 0.9808125457698592, 0.9962276403335677, 0.9918168035760903, 0.9985037211943377, 0.9807816030842715, 0.9842896369179833, 0.9906214993263324, 0.99746291

In [ ]:
unlabel_1=unlabel_1.loc[pos,:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
2,525,85 22 sw 140 wherein accused acting upon advic...,Rule/Law/Holding
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts
8,944,relies upon tonnahill v,Invalid


In [ ]:
frame_1 = [train_1,unlabel_1]
train_1_1 = pd.concat(frame_1)
len(train_1_1)

3533

In [ ]:
x_train_1 = tfidf_vect.transform(train_1_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train_1_1['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6647807637906648
                  precision    recall  f1-score   support

        Analysis       0.46      0.32      0.38        97
      Conclusion       0.74      0.49      0.59        35
           Facts       0.72      0.89      0.80       369
         Invalid       0.57      0.53      0.55       105
           Issue       0.60      0.39      0.47        46
Rule/Law/Holding       0.64      0.33      0.43        55

        accuracy                           0.66       707
       macro avg       0.62      0.49      0.54       707
    weighted avg       0.65      0.66      0.64       707



In [ ]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, 3, 1, 2, 2, 2, 1, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 2, 3, 3, 2, 3, 2, 2, 2, 2, 2, 5, 2]
[0.9985853672663101, 0.999021626697899, 0.9898398170044909, 0.9920969376907213, 0.9980285980382733, 0.9986334540849929, 0.9976926398636792, 0.9989560629728871, 0.9985795621177026, 0.9866783291441678, 0.997768215113809, 0.9850883263759492, 0.9913530662644473, 0.9980958694017261, 0.9945913912365318, 0.9954567399940666, 0.9891144918879833, 0.9893829878329058, 0.9975537908103602, 0.9989647045333889, 0.9990062202636112, 0.9881122887550612, 0.998759223807701, 0.9945590908652719, 0.9976429170101094, 0.9984974959457533, 0.9948366980347204, 0.994129903258849, 0.9988366856108037, 0.9878004922293154, 0.9910812004981289, 0.9987373018648131, 0.9989922382165739, 0.9807187636365233, 0.997575527037756, 0.99870911929497, 0.9875570060993846, 0.9986299240102589, 0.9923713386361854, 0.9973691705237065, 0.9988192074854075, 0.9932659550086683, 0.9989978042729

In [ ]:
unlabel_2 = unlabel_2.loc[pos,:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
5,2278,evidence state appellant shows appellant alleg...,Analysis
6,570,appellant performed work asked 7500 money ther...,Facts


In [ ]:
frame_2 = [train_1_1,unlabel_2]
train1_2 = pd.concat(frame_2)
len(train1_2)

3583

In [ ]:
x_train_2 = tfidf_vect.transform(train1_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train1_2['Target'],test_size=0.27,random_state=2)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6260330578512396
                  precision    recall  f1-score   support

        Analysis       0.48      0.30      0.37       152
      Conclusion       0.52      0.30      0.38        50
           Facts       0.68      0.88      0.77       495
         Invalid       0.49      0.51      0.50       136
           Issue       0.74      0.29      0.42        69
Rule/Law/Holding       0.61      0.35      0.44        66

        accuracy                           0.63       968
       macro avg       0.58      0.44      0.48       968
    weighted avg       0.61      0.63      0.60       968



In [ ]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 3, 3, 2, 2, 3, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 1, 3, 3, 1, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2]
[0.9932357260304155, 0.9963629464565846, 0.9979551803326471, 0.9981927093278543, 0.986478223942795, 0.9972661972047088, 0.9982229872204182, 0.99859962186495, 0.9964197597786257, 0.984608433433022, 0.9907537285284259, 0.9986551062276532, 0.9983333941976305, 0.9986711976111011, 0.998759728263968, 0.9926631297990993, 0.9983031827779819, 0.9978468777483784, 0.9899718340495895, 0.9982878132367005, 0.9970099229988806, 0.9985059199927176, 0.9895233171891289, 0.9984103797574303, 0.9823820823541333, 0.9906901762288157, 0.9956986106767673, 0.9975432368289963, 0.9974932069880821, 0.9956557075096193, 0.9953329140451933, 0.9877195500618277, 0.9981701607750878, 0.9974663477400358, 0.9820753900053834, 0.9983385877126865, 0.99291369110077, 0.9981693894269048, 0.9934794569826124, 0.9982615005266836, 0.98279453223236

In [ ]:
unlabel_3=unlabel_3.loc[pos,:]
frame_3 = [train1_2,unlabel_3]
train1_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train1_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train1_3['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6589041095890411
                  precision    recall  f1-score   support

        Analysis       0.44      0.28      0.34        95
      Conclusion       0.69      0.44      0.54        45
           Facts       0.70      0.88      0.78       384
         Invalid       0.58      0.56      0.57       105
           Issue       0.62      0.41      0.49        51
Rule/Law/Holding       0.69      0.36      0.47        50

        accuracy                           0.66       730
       macro avg       0.62      0.49      0.53       730
    weighted avg       0.64      0.66      0.64       730



In [ ]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[3, 2, 3, 2, 5, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 3, 2, 2, 0, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 3, 2, 2, 0, 2, 2, 3, 2, 2, 5, 3, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2]
[0.9970754231841054, 0.9952558253338317, 0.9939199953709448, 0.9979535251902591, 0.9977706066190607, 0.9989427240254202, 0.9931709302406571, 0.9875415436308709, 0.9989856888816429, 0.9988373454691499, 0.9918843729512286, 0.9856818284327621, 0.9983634772798148, 0.9985112270859718, 0.9989556367090902, 0.9987800897991175, 0.9975515271798148, 0.9984676012902527, 0.9960168008005481, 0.9868489094732703, 0.9846990240764324, 0.998639196621454, 0.9989289151725701, 0.9988219387231735, 0.9988238952351008, 0.9982830385559374, 0.9987807630341107, 0.9988825744857215, 0.9978319984640838, 0.9983133507610208, 0.9817245443764814, 0.9962324253683003, 0.9985705290244475, 0.990606607572521, 0.9971370831169786, 0.9970472080318348, 0.9939061949212178, 0.9904288412868525, 0.9883348781672945, 0.9941736622381011, 0.996958002608471

In [ ]:
unlabel_4=unlabel_4.loc[pos,:]

In [ ]:
frame_4 = [train1_3,unlabel_4]
train1_4 = pd.concat(frame_4)
len(train1_4)
x_train_4 = tfidf_vect.transform(train1_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train1_4['Target'],test_size=0.27,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6373626373626373
                  precision    recall  f1-score   support

        Analysis       0.48      0.35      0.40       142
      Conclusion       0.70      0.42      0.52        55
           Facts       0.72      0.85      0.78       531
         Invalid       0.44      0.50      0.47       144
           Issue       0.62      0.42      0.50        57
Rule/Law/Holding       0.61      0.28      0.38        72

        accuracy                           0.64      1001
       macro avg       0.59      0.47      0.51      1001
    weighted avg       0.63      0.64      0.62      1001



In [ ]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 3, 3, 2, 3, 0, 4, 2, 5, 3, 3, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 0, 2, 2, 0, 2, 2, 2, 3, 3, 2, 3, 2, 2, 2, 1, 2, 2, 2, 3, 2, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 0, 2, 0, 3, 2, 2]
[0.9852407436138595, 0.9988296941294352, 0.9909309966606376, 0.9977894766252953, 0.9864495015270062, 0.9979891544345917, 0.987672441715983, 0.9806853372330044, 0.9978371348231976, 0.9900036664208565, 0.9982723577360986, 0.9971097630801873, 0.9977096312895986, 0.9980568680111467, 0.9831614384471928, 0.9945342544856868, 0.9893693097925101, 0.9904424184865099, 0.9914954830168694, 0.9987210353877696, 0.9987908860973408, 0.9921544472654448, 0.9967888427098862, 0.9812621598862515, 0.989359898531649, 0.981484674950963, 0.9842753222554682, 0.9966432996053745, 0.9985969919447052, 0.99865724523418, 0.9980282825295905, 0.9977894766252953, 0.9948698260192366, 0.9983953129540555, 0.9966207062251718, 0.9987158899105792, 0.996238527456417, 0.9987210353877696, 0.9881040443999483, 0.9940305493960622, 0.9962550169

In [ ]:
unlabel_5 = unlabel_5.loc[pos,:]

In [ ]:
frame_5 = [train1_4,unlabel_5]
train1_5 = pd.concat(frame_5)
len(train1_5)
x_train_5 = tfidf_vect.transform(train1_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train1_5['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6635761589403973
                  precision    recall  f1-score   support

        Analysis       0.58      0.35      0.44       116
      Conclusion       0.75      0.45      0.56        40
           Facts       0.71      0.90      0.79       387
         Invalid       0.52      0.58      0.55       100
           Issue       0.66      0.43      0.52        49
Rule/Law/Holding       0.58      0.24      0.34        63

        accuracy                           0.66       755
       macro avg       0.63      0.49      0.53       755
    weighted avg       0.65      0.66      0.64       755



In [ ]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.7428023032629558


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(test['Target'],test_pred,average='macro'))

(0.7275463783528299, 0.6212126259228083, 0.6553840192729082, None)
